In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(font_scale=1.5)
sns.set_style("whitegrid")

def normalize(num, sourceMin, sourceMax, targetMin, targetMax):
    sourceRange = sourceMax-sourceMin;
    targetRange = targetMax-targetMin;
    return (num-sourceMin)*(targetRange/sourceRange)+targetMin

In [2]:
#get the list of cities and Lat Long, also population number from the dataset in csv format
#the data is acquired from https://worldpopulationreview.com/countries/cities/indonesia
df = pd.read_csv("indonesia_cities.csv")
df.shape

(384, 4)

In [3]:
df.head()

,City,Population,Lat,Lng
0,Jakarta,8540121,-6.21462,106.8450
1,Surabaya,2374658,-7.24917,112.7510
2,Medan,1750971,3.58333,98.6667
3,Bandung,1699719,-6.92222,107.6070
4,Bekasi,1520119,-6.23490,106.9900


In [17]:
#initialization for folium
import folium
import matplotlib.colors as colors
def foliumMark(row,color):
    return folium.CircleMarker(
        [row["Lat"], row["Lng"]],
        radius=2,
        popup=str(row["City"]),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7)

In [18]:
#now show the map and the markers
#the center location is Jakarta
map = folium.Map(location=[-6.21462,106.84513], zoom_start=4)
for index,row in df.iterrows():
    color = "#6060ff"
    foliumMark(row,color).add_to(map)
map

In [44]:
#get the number of recommended venues in each city using foursquare API
import json
import collections
from urllib import request

def getVenues(lat,lng):
    apiresult = request.urlopen(
        "https://api.foursquare.com/v2/venues/search?v=20210126&client_id=IKE5IUMRJQOFKU4OPT5VGWE434RGHUV1CCYFLUY2ODYYQE1F&client_secret=E4ORZYBW5S3DSSG2YXZBUEM1B23332NXNMWZL30H2TN405VL&ll="+str(lat)+","+str(lng)
    ).read()
    res = json.loads(apiresult)
    return res["response"]["venues"]

def parseCategories(venues):
    #get the category names in array, can be duplicated
    cname = []
    for venue in venues:
        cat_arr = venue["categories"]
        for cat in cat_arr:
            cname.append(cat["name"])
    return cname
    
#put the number of venues in the df
df["Num Venues"] = 0
cnames = [] #for all cities
cnames_percity = {} #for each city
for index,row in df.iterrows():
    city = df.loc[index,"City"]
    venues = getVenues(row["Lat"],row["Lng"])
    categories = parseCategories(venues)
    #put per city
    cnames_percity[city] = collections.Counter(categories)
    #put it globally
    cnames = cnames+categories
    df.loc[index,"Num Venues"]=len(venues)
cnames = collections.Counter(cnames)
#print(cnames_percity["Jakarta"]["University"])
#at this point, we get the frequencies of categories from all venues from all cities
#e.g. Counter({'University': 12, 'College Library': 11})

In [106]:
#show the frequencies
print(cnames)

Counter({'Office': 388, 'Indonesian Restaurant': 377, 'Government Building': 234, 'Building': 219, 'Asian Restaurant': 191, 'Bank': 173, 'Mosque': 161, 'Hospital': 107, 'Café': 102, 'Church': 101, 'Coffee Shop': 97, 'Medical Center': 84, 'Diner': 80, 'Convenience Store': 79, 'Capitol Building': 74, 'Hotel': 72, 'Student Center': 71, 'Automotive Shop': 68, "Doctor's Office": 68, 'Department Store': 67, 'Food Truck': 64, 'Salon / Barbershop': 60, 'Factory': 59, 'Gas Station': 56, 'Restaurant': 53, 'Other Great Outdoors': 50, 'Chinese Restaurant': 50, 'College Classroom': 48, 'Pharmacy': 48, 'Electronics Store': 46, 'Miscellaneous Shop': 46, 'Coworking Space': 45, 'Bakery': 45, 'High School': 44, 'Post Office': 44, 'Noodle House': 44, "Dentist's Office": 43, 'Breakfast Spot': 43, 'Police Station': 43, 'Housing Development': 42, 'Park': 41, 'Market': 41, 'Arcade': 37, 'Soup Place': 37, 'Fast Food Restaurant': 36, 'Residential Building (Apartment / Condo)': 36, 'City Hall': 34, 'Courthouse'

In [47]:
print(df.head())

       City  Population      Lat       Lng  Num Venues  Cluster
0   Jakarta     8540121 -6.21462  106.8450          30        0
1  Surabaya     2374658 -7.24917  112.7510          30        3
2     Medan     1750971  3.58333   98.6667          30        2
3   Bandung     1699719 -6.92222  107.6070          30        0
4    Bekasi     1520119 -6.23490  106.9900          30        0


In [52]:
#cluster the cities into 10
kclusters = 10
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=kclusters)
kmeans.fit(df[["Lat","Lng"]])
kmeans.labels_

array([4, 8, 1, 4, 4, 9, 4, 3, 4, 0, 4, 9, 9, 8, 4, 4, 8, 9, 3, 0, 8, 3,
       0, 4, 6, 2, 3, 9, 2, 3, 8, 1, 9, 8, 3, 5, 4, 4, 0, 4, 1, 0, 8, 1,
       0, 4, 3, 1, 4, 4, 4, 4, 4, 3, 4, 9, 4, 5, 4, 0, 0, 4, 8, 4, 4, 0,
       4, 4, 4, 8, 8, 1, 1, 4, 4, 0, 4, 9, 6, 4, 4, 2, 1, 1, 3, 8, 4, 7,
       9, 0, 8, 8, 3, 1, 3, 4, 0, 4, 0, 8, 4, 9, 2, 4, 4, 8, 4, 0, 9, 8,
       4, 1, 0, 0, 4, 4, 8, 4, 0, 4, 4, 8, 4, 4, 0, 9, 1, 1, 6, 2, 3, 0,
       1, 4, 9, 0, 3, 9, 4, 0, 4, 8, 4, 9, 0, 4, 8, 1, 4, 0, 4, 4, 0, 0,
       8, 0, 4, 3, 3, 8, 4, 1, 8, 8, 0, 4, 5, 8, 8, 4, 0, 8, 9, 8, 0, 4,
       4, 0, 9, 0, 8, 8, 0, 5, 0, 0, 6, 9, 8, 8, 8, 0, 6, 7, 8, 0, 0, 8,
       3, 0, 3, 8, 0, 0, 0, 3, 4, 0, 8, 1, 0, 8, 9, 9, 0, 0, 3, 8, 0, 0,
       1, 8, 7, 9, 5, 4, 8, 0, 0, 0, 4, 0, 1, 8, 5, 0, 4, 5, 0, 9, 4, 8,
       2, 6, 2, 8, 5, 3, 1, 9, 0, 0, 1, 0, 0, 0, 7, 0, 8, 4, 8, 5, 0, 4,
       8, 9, 3, 3, 1, 0, 1, 4, 7, 0, 0, 0, 8, 0, 6, 3, 0, 4, 9, 8, 2, 5,
       3, 0, 8, 8, 1, 5, 6, 8, 6, 9, 8, 0, 4, 0, 2,

In [93]:
#create colors from green to red
#green indicates the best potential cluster for business, with the lowest value of venue/population
from colour import Color
colors = list(Color("#4FAD4F").range_to(Color("#DF5C24"),100))

In [54]:
#put the cluster labels in the df
df["Cluster"] = 0
for index,row in df.iterrows():
    df.loc[index,"Cluster"]=kmeans.labels_[index]
df.head()

,City,Population,Lat,Lng,Num Venues,Cluster
0,Jakarta,8540121,-6.21462,106.8450,30,4
1,Surabaya,2374658,-7.24917,112.7510,30,8
2,Medan,1750971,3.58333,98.6667,30,1
3,Bandung,1699719,-6.92222,107.6070,30,4
4,Bekasi,1520119,-6.23490,106.9900,30,4


In [108]:
#calculate VP for each cluster (venue/population)
#lower VP indicates that there are still a few number of venues if compared to the population count in that cluster
vp = pd.DataFrame(data={"Cluster":np.arange(0,kclusters)})
vp["Venues"]=""
vp["Population"]=""
vp["Cities"]=0
vp["VP"]=""
for i in range(0,kclusters):
    vsum = df[df["Cluster"]==i]["Num Venues"].sum()
    psum = df[df["Cluster"]==i]["Population"].sum()
    vp.loc[i,"Population"] = psum
    vp.loc[i,"Venues"] = vsum
    vp.loc[i,"Cities"] = df[df["Cluster"]==i]["City"].count()
    #VP in percent
    vp.loc[i,"VP"] = 100*vsum/psum

In [110]:
#now redraw the map with the clusters and the green color for low VP, and red color for high VP
#the center location is Jakarta
map = folium.Map(location=[-6.21462,106.84513], zoom_start=4)
for index,row in df.iterrows():
    #get the color from the VP table
    vp_value = vp[vp["Cluster"]==row["Cluster"]].iloc[0]["VP"]
    color = str(colors[int(normalize(vp_value,vp["VP"].min(),vp["VP"].max(),0,99))])
    foliumMark(row,color).add_to(map)
map

This map clearly shows the green areas where there's still a lot of potential to open new business, since the number of venues are still low if compared to the population.
<br>On the other hand, red areas have already a lot of venues if compared to population.
<br>Thus, for example, one potential area is West Java area.

Do further analysis by coloring the markers based on business categories
We'll take 10 most popular business categories and count the frequency of these categories by cluster

In [62]:
#take a look at the categories
#print(cnames)
#we'll take top 5 profitable business categories :
categ_use = ["Indonesian Restaurant","Asian Restaurant","Café","Coffee Shop","Convenience Store"]
for cat in categ_use:
    print("Freq for "+str(cat)+" = "+str(cnames[cat]))

Freq for Indonesian Restaurant = 377
Freq for Asian Restaurant = 191
Freq for Café = 102
Freq for Coffee Shop = 97
Freq for Convenience Store = 79


In [109]:
#embed the frequencies of businesses to vp (clusters dataframe)
for cat in categ_use:
    vp[cat] = 0
    for i in range(0,kclusters):
        df_incluster = df[df["Cluster"]==i]
        for index,row in df_incluster.iterrows():
            city = row["City"]
            vp.loc[i,cat] = vp.loc[i,cat]+cnames_percity[city][cat]
vp

,Cluster,Venues,Population,Cities,VP,Indonesian Restaurant,Asian Restaurant,Café,Coffee Shop,Convenience Store
0,0,2847,8559782,95,0.0332602,94,53,18,19,19
1,1,1106,4724106,37,0.0234118,37,22,16,19,5
2,2,360,1389519,12,0.0259082,14,5,2,3,2
3,3,780,5318560,26,0.0146656,34,7,7,9,3
4,4,2190,25607290,73,0.00855225,70,30,10,10,18
5,5,370,983694,13,0.0376133,19,10,7,1,2
6,6,300,1006806,10,0.0297972,4,8,3,4,3
7,7,150,333733,5,0.0449461,0,0,3,0,1
8,8,2501,9761830,84,0.0256202,93,41,29,26,23
9,9,820,6639103,29,0.0123511,12,14,7,5,3


In [104]:
#now redraw the map with the clusters and the green color for places with less venues, red color for places with high venues
for field in categ_use:
    print("Category "+field)
    f = folium.Figure(width=600, height=300)
    map = folium.Map(location=[-0.49167,117.146], zoom_start=4).add_to(f)
    for index,row in df.iterrows():
        num_venues = vp[vp["Cluster"]==row["Cluster"]].iloc[0][field]
        color = str(colors[int(normalize(num_venues,vp[field].min(),vp[field].max(),0,99))])
        foliumMark(row,color).add_to(map)
    display(f)
    print("-------")

Category Indonesian Restaurant


-------
Category Asian Restaurant


-------
Category Café


-------
Category Coffee Shop


-------
Category Convenience Store


-------
